### Importy

In [86]:
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta as tmd

from warnings import filterwarnings
filterwarnings('ignore')

from importlib import reload
import DataLoader.xAPIConnector
reload(DataLoader.xAPIConnector)
from DataLoader.xAPIConnector import *


import DataLoader.DataLoader
reload(DataLoader.DataLoader)
from DataLoader.DataLoader import *

from DataLoader.config import user_id, pwd

# Pobieranie danych

Należy podać listę symboli, datę, od której chcemy zaciągnąć dane i częstotliwość (teraz '5min'). Można też podać datę 'end', ale domyślnie zaciąga się do chwili obecnej.

In [16]:
symbols = ['BITCOIN', 'ETHEREUM']
start, interval = '2024-12-01 00:00:00', '5min'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval)

[2025-01-14 11:46:43.372569] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-14 11:46:50.603566] Wylogowuję z API...


Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Pierwszy pomysł to klasyfikacja na podstawie stóp zwrotu, klasy również będziemy budować na podstawie stóp zwrotu. Metoda budowania klas zostanie omówiona dalej.

In [106]:
window = int(60/int(interval[:-3]))
returnsBTC = data.loc[:, 'BITCOIN'].pct_change().dropna()

In [141]:
def prepareData(data: pd.Series, window: int, skip: int) -> tuple:   
    # można ustawić okna zachodzące (0 < skip < window)
    # można ustawić niezachodzące (skip >= window)
    assert skip > 0, "Pętla w kodzie nigdy się nie zakończy..."
    
    # Generujemy 'okna'
    X = pd.DataFrame(columns=range(window))
    
    i = len(data)
    count = 0
    while i >= window:
        temp_y = data.iloc[i-window:i]
        
        X.loc[count, :] = temp_y.values
        
        i = i - skip    
        count += 1
        
    return np.array(X)

# Modele klasyfikacyjne

In [156]:
from sklearn.linear_model import LogisticRegression as LogR, LinearRegression as LinR
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import GradientBoostingClassifier as GBC

### Definiowanie klas

Pierwszy pomysł na zdefiniowanie klas to przyjęcie $1$ jeśli stopa zwrotu jest $>$'threshold' oraz $0$ w p.p.

Próg odcięcia można podać jako parametr i już po pierwszych testach widać, że dla prostej klasyfikacji typu "czy zwrot > $0$" skuteczność jest około $50\%$. Im wyższy próg, tym lepsza skuteczność.

In [135]:
skip = 1
arr = prepareData(data=returnsBTC, window=window, skip=skip)
X = arr[:, :-1]
y = arr[:, -1]

threshold = np.quantile(y, 0.75)
y = (y > threshold)*1.0

for i in range(0, 2):
    print(f"Klasa {i}: {np.sum(y == i)/len(y):.4%}")

Klasa 0: 75.0027%
Klasa 1: 24.9973%


Teraz podzielimy dane na treningowe i testowe, po czym sprawdzimy jakość predykcji

In [136]:
train_test_ratio = 0.7
len_train = int(train_test_ratio*X.shape[0])

Xtrain = X[:len_train, :]
ytrain = y[:len_train]

Xtest = X[len_train:, :]
ytest = y[len_train:]

print(f"[INFO] W zbiorze treningowym znaduje się {Xtrain.shape[0]} obserwacji.")
print(f"[INFO] W zbiorze testowym znajduje się {Xtest.shape[0]} obserwacji.")

[INFO] W zbiorze treningowym znaduje się 6527 obserwacji.
[INFO] W zbiorze testowym znajduje się 2798 obserwacji.


In [ ]:
print(f"[INFO] Regresja logistyczna")
clf = LogR().fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Regresja logistyczna
	Skuteczność treningowa modelu: 76.3138%
	Skuteczność testowa modelu: 71.9442%


In [138]:
print(f"[INFO] Support Vector Machines")
clf = SVC(kernel='poly').fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Support Vector Machines
	Skuteczność treningowa modelu: 77.9838%
	Skuteczność testowa modelu: 70.8006%


In [139]:
print(f"[INFO] Drzewo decyzyjne")
clf = DTC(max_depth=3).fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Drzewo decyzyjne
	Skuteczność treningowa modelu: 76.6662%
	Skuteczność testowa modelu: 71.8013%


In [140]:
print(f"[INFO] Las decyzyjny")
clf = GBC(n_estimators=100, max_depth=5).fit(Xtrain, ytrain)
print(f"\tSkuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"\tSkuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

[INFO] Las decyzyjny
	Skuteczność treningowa modelu: 83.9896%
	Skuteczność testowa modelu: 69.0136%


Predykcje uzyskane przy użyciu tych metod tworzenia $y$ dają rezultaty gorsze od naiwnego klasyfikatora przypisującego zawsze $0$.

### Inne pomysły na definiowanie klas

Pomysł oparty na regresji liniowej

$$y_t = \beta_0 + \beta_{-5}y_{t-5} + ... + \beta_{-1}y_{t-1} + \beta_1y_{t+1} + ... +\beta_5y_{t+5}$$

In [161]:
window = 11
skip = window

arr = prepareData(data=returnsBTC, window=window, skip=skip)
X = np.delete(arr, 5, axis=1)
y = arr[:, 5]

In [162]:
model = LinR().fit(X, y)
model.coef_

array([ 0.01991393, -0.04023283, -0.03274852,  0.05045935,  0.02898242,
       -0.08376527, -0.04969022,  0.0946578 , -0.00211609,  0.07973739])

In [159]:
np.mean(model.coef_)

0.006519796843450826

# Model na akcjach spółek surowcowych

## Strategia trend-following

In [62]:
def position(x):
    if x[1] == True: return 1
    elif x[0] == True and x[1] == False: return -1
    elif x[0] == False and x[1] == False: return 0
    
def EMA(y: pd.Series, alpha: float):
    weights = np.array([(1-alpha)**(len(y)-i+1) for i in range(len(y))])
    return (y.values).dot(weights)/np.sum(weights)

In [77]:
import json

with open('spolki.json') as f:
    data = json.load(f)

In [81]:
gold_symbols = [key+'.US' for key in data['gold'].keys()]
gold_symbols

['NEM.US',
 'AU.US',
 'RGLD.US',
 'CDE.US',
 'SA.US',
 'CMCL.US',
 'IDR.US',
 'CTGO.US',
 'FTCO.US',
 'USAU.US']

In [87]:
symbols = gold_symbols
start, interval = '2020-01-01 00:00:00', '1D'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval)

[2025-01-16 15:21:36.526877] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-16 15:21:36.956476] Wylogowuję z API...
[2025-01-16 15:21:37.107886] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-16 15:21:37.745759] Wylogowuję z API...
[2025-01-16 15:21:37.827176] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-16 15:21:38.330104] Wylogowuję z API...
[2025-01-16 15:21:38.406924] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano CDE.US
[2025-01-16 15:21:39.039580] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano SA.US
[2025-01-16 15:21:39.409580] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano CMCL.US
[2025-01-16 15:21:39.797431] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[BŁĄD] Nie pobrano IDR.US
[2025-01-16 15:21:40.

In [88]:
data

,NEM.US,AU.US,RGLD.US
Date,,,
2020-01-02,43.25,22.64,120.80
2020-01-03,42.73,22.49,119.94
2020-01-06,43.21,22.62,118.32
2020-01-07,43.24,22.43,120.01
2020-01-08,41.98,21.44,111.02
...,...,...,...
2025-01-08,38.89,25.08,135.70
2025-01-10,38.85,25.86,133.62
2025-01-13,39.05,25.44,133.52


In [105]:
ans = pd.Series(pd.NA, index = data.columns, name='TotalReturn')

for symbol in data.columns:
    y = data.loc[:, symbol]
    Returns = y.pct_change()
    MAshort = y.rolling(10).apply(lambda x: EMA(x, 0.1))
    MAlong = y.rolling(22).apply(lambda x: EMA(x, 0.1))

    UpwardTrend = (MAshort > MAlong)
    Decision = UpwardTrend.rolling(2).apply(lambda x: position(x))

    TotalReturn = (1 + Returns[Decision == 1.0]).prod() - 1
    ans[symbol] = TotalReturn
    print(f"{symbol}: skumulowany zwrot ze strategii opartej na porównywaniu MA: {TotalReturn:.2%}")

NEM.US: skumulowany zwrot ze strategii opartej na porównywaniu MA: 103.01%
AU.US: skumulowany zwrot ze strategii opartej na porównywaniu MA: 101.33%
RGLD.US: skumulowany zwrot ze strategii opartej na porównywaniu MA: 172.01%


In [106]:
ans

NEM.US     1.030149
AU.US      1.013333
RGLD.US    1.720089
Name: TotalReturn, dtype: object